In [1]:
from data import *
from utils.augmentations import SSDAugmentation
from layers.modules import MultiBoxLoss
from ssd import build_ssd
import os
import sys
import time
import torch
from torch.autograd import Variable
import torch.nn as nn

import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torch.utils.data as data
import numpy as np
import argparse

ImportError: No module named 'data'

## Argument Declare

In [2]:
DATASET_ROOT = "/media/arg_ws3/5E703E3A703E18EB/data/subt_real"
DATASET_NAME = "subt_artifact"
cfg = subt
BASE_NET = "./weights/vgg16_reducedfc.pth"
DATA_DETECTION = SUBTDetection
BATCH_SIZE = 32
#PRETRAINED_MODEL = "/home/arg_ws3/ssd.pytorch/weights/person/person_67500.pth"
PRETRAINED_MODEL = None
PRETRAINED_EPOCH = 0
SAVE_MODEL_EPOCH = 5
START_ITER = 0
NUM_WORKERS = 8
EPOCH = 61
adjust_lr_epoch = [15, 30, 45]
CUDA = True
LR = 1e-4
MOMENTUM = 0.9
WEIGHT_DECAY = 5e-4
GAMMA = 0.1
VISDOM = False
SAVE_FOLDER = "/media/arg_ws3/5E703E3A703E18EB/data/subt_real/model/"
if not os.path.exists(SAVE_FOLDER):
    os.makedirs(SAVE_FOLDER)

In [3]:
if torch.cuda.is_available():
    if not CUDA:
        print("WTF are u wasting your CUDA device?")
    else:
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')

## Define function

In [4]:
def str2bool(v):
    return v.lower() in ("yes", "true", "t", "1")

# Initial model weights & bias
def xavier(param):
    init.xavier_uniform(param)
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        xavier(m.weight.data)
        m.bias.data.zero_()

# Adjust learning rate during training
def adjust_learning_rate(optimizer, gamma, step):
    """Sets the learning rate to the initial LR decayed by 10 at every
        specified step 
    # Adapted from PyTorch Imagenet example:
    # https://github.com/pytorch/examples/blob/master/imagenet/main.py
    """
    #lr = LR * (gamma ** (step))
    lr = LR * (gamma)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
        print("Change learning rate to: ", lr)

## Create dataset

In [5]:
dataset = DATA_DETECTION(root=DATASET_ROOT, image_sets=['train'],transform=SSDAugmentation(cfg['min_dim'], MEANS))

classes = dataset.target_transform.class_to_ind
print("Class to index: \n", classes)
classes = sorted(classes.items(), key=lambda kv: kv[1])
label = []
for i in classes:
    label.append(i[0])
label.append('None')
print(label)
true_label = ['person', 'palm']

Class to index: 
 {'bb_extinguisher': 0, 'bb_drill': 1}
['bb_extinguisher', 'bb_drill', 'None']


## Create model

In [6]:
ssd_pretrained = build_ssd('train', cfg['min_dim'], 3)
if CUDA:
    net = torch.nn.DataParallel(ssd_pretrained)
    cudnn.benchmark = True
    
if PRETRAINED_MODEL is not None: # Use SSD pretrained model
    print('Resuming training, loading {}...'.format(PRETRAINED_MODEL))
    ssd_pretrained.load_weights(PRETRAINED_MODEL)

/home/arg_ws3/ssd.pytorch/ssd.py:34: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.priors = Variable(self.priorbox.forward(), volatile=True)
/home/arg_ws3/ssd.pytorch/layers/modules/l2norm.py:17: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(self.weight,self.gamma)


In [7]:
# Delcare SSD Network
ssd_net = build_ssd('train', cfg['min_dim'], cfg['num_classes'])
net = ssd_net
if CUDA:
    net = torch.nn.DataParallel(ssd_net)
    cudnn.benchmark = True
SAME_CLASS = False
if PRETRAINED_MODEL is not None: # Use SSD pretrained model
    if SAME_CLASS:
        print('Resuming training, loading {}...'.format(PRETRAINED_MODEL))
        ssd_net.load_weights(PRETRAINED_MODEL)
    else:
        print('Load pretrained model with different classes')
        ssd_pretrained = build_ssd('train', cfg['min_dim'], 2)
        ssd_pretrained.load_weights(PRETRAINED_MODEL)
        ssd_net.vgg = ssd_pretrained.vgg
        ssd_net.extras = ssd_pretrained.extras
        ssd_net.loc = ssd_pretrained.loc
        ssd_net.conf.apply(weights_init)
else:
    print('Initializing weights...')
    vgg_weights = torch.load(BASE_NET) # load vgg pretrained model
    ssd_net.vgg.load_state_dict(vgg_weights)
    ssd_net.extras.apply(weights_init) # Initial SSD model weights & bias
    ssd_net.loc.apply(weights_init)
    ssd_net.conf.apply(weights_init)

optimizer = optim.SGD(net.parameters(), lr=LR, momentum=MOMENTUM,
                weight_decay=WEIGHT_DECAY)
criterion = MultiBoxLoss(BATCH_SIZE ,cfg['num_classes'], 0.5, True, 0, True, 3, 0.5,
                False, CUDA)

Initializing weights...


/home/arg_ws3/.local/lib/python3.5/site-packages/ipykernel_launcher.py:6: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  


In [8]:
#for i in range(len(list(ssd_net.parameters()))-30):
#    list(ssd_net.parameters())[i].data = list(ssd_pretrained.parameters())[i]
#list(ssd_net.conf)[-1] = list(ssd_pretrained.conf)[-1]

## Training

### Initialize

In [9]:
net.train()
# loss counters
loc_loss = 0
conf_loss = 0
epoch = 0
print('Loading the dataset...')
epoch_size = len(dataset) // BATCH_SIZE
print('Training SSD on:', DATASET_NAME)

data_loader = data.DataLoader(dataset, BATCH_SIZE,
                                num_workers=NUM_WORKERS,
                                shuffle=True, collate_fn=detection_collate,
                                pin_memory=True)
batch_iterator = iter(data_loader)

Loading the dataset...
Training SSD on: subt_artifact


### Start training

In [10]:
iteration = 0
for epoch in range(EPOCH):
    for i, batch in enumerate(data_loader):
        iteration += 1
        images, targets = batch
        if CUDA:
            images = Variable(images.cuda())
            targets = [Variable(ann.cuda(), volatile=True) for ann in targets]
        else:
            images = Variable(images)
            targets = [Variable(ann, volatile=True) for ann in targets]

        # Forward
        t0 = time.time()
        out = net(images)
        # backprop
        optimizer.zero_grad()
        loss_l, loss_c = criterion(out, targets)
        loss = loss_l + loss_c
        loss.backward()
        optimizer.step()
        t1 = time.time()
        loc_loss += loss_l.item()
        conf_loss += loss_c.item()

        if iteration % 8 == 0:
                print('timer: %.4f sec.' % (t1 - t0))
                print('Epoch: ' + str(epoch) + ' || iter ' + repr(PRETRAINED_EPOCH + iteration) + ' || Loss: %.4f ||' % (loss.item()), end='')

    if epoch != 0 and epoch % SAVE_MODEL_EPOCH == 0:
        print('Saving state, Epoch:', epoch)
        torch.save(ssd_net.state_dict(), SAVE_FOLDER + DATASET_NAME + "_" +
                    repr(PRETRAINED_EPOCH + epoch) + '.pth')
    if epoch in adjust_lr_epoch:
        adjust_learning_rate(optimizer, GAMMA, epoch)
    # Save final model
torch.save(ssd_net.state_dict(),
        SAVE_FOLDER + DATASET_NAME + '.pth')

/home/arg_ws3/.local/lib/python3.5/site-packages/ipykernel_launcher.py:8: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  
/home/arg_ws3/.local/lib/python3.5/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


timer: 0.3040 sec.
Epoch: 0 || iter 8 || Loss: 14.1264 ||timer: 0.3065 sec.
Epoch: 0 || iter 16 || Loss: 8.9240 ||timer: 0.3063 sec.
Epoch: 0 || iter 24 || Loss: 8.0102 ||timer: 0.3316 sec.
Epoch: 0 || iter 32 || Loss: 7.7912 ||timer: 0.3311 sec.
Epoch: 0 || iter 40 || Loss: 7.1676 ||timer: 0.3321 sec.
Epoch: 0 || iter 48 || Loss: 6.9750 ||timer: 0.3316 sec.
Epoch: 0 || iter 56 || Loss: 6.9701 ||timer: 0.3307 sec.
Epoch: 0 || iter 64 || Loss: 6.7572 ||timer: 0.3307 sec.
Epoch: 0 || iter 72 || Loss: 6.2997 ||timer: 0.3062 sec.
Epoch: 1 || iter 80 || Loss: 6.3721 ||timer: 0.3064 sec.
Epoch: 1 || iter 88 || Loss: 6.2638 ||timer: 0.3351 sec.
Epoch: 1 || iter 96 || Loss: 6.4558 ||timer: 0.3324 sec.
Epoch: 1 || iter 104 || Loss: 5.9591 ||timer: 0.3315 sec.
Epoch: 1 || iter 112 || Loss: 6.1522 ||timer: 0.3324 sec.
Epoch: 1 || iter 120 || Loss: 6.2604 ||timer: 0.3301 sec.
Epoch: 1 || iter 128 || Loss: 6.2278 ||timer: 0.3295 sec.
Epoch: 1 || iter 136 || Loss: 5.8336 ||timer: 0.3308 sec.
Epoch: 

Epoch: 30 || iter 2200 || Loss: 1.9727 ||timer: 0.3314 sec.
Epoch: 30 || iter 2208 || Loss: 1.7055 ||timer: 0.3333 sec.
Epoch: 30 || iter 2216 || Loss: 1.8683 ||timer: 0.3337 sec.
Epoch: 30 || iter 2224 || Loss: 2.1711 ||timer: 0.3332 sec.
Epoch: 30 || iter 2232 || Loss: 2.0588 ||timer: 0.3325 sec.
Epoch: 30 || iter 2240 || Loss: 1.9905 ||timer: 0.3306 sec.
Epoch: 30 || iter 2248 || Loss: 1.9505 ||timer: 0.3294 sec.
Epoch: 30 || iter 2256 || Loss: 2.0431 ||Saving state, Epoch: 30
Change learning rate to:  1e-05
timer: 0.3784 sec.
Epoch: 31 || iter 2264 || Loss: 2.2902 ||timer: 0.3307 sec.
Epoch: 31 || iter 2272 || Loss: 2.0899 ||timer: 0.3329 sec.
Epoch: 31 || iter 2280 || Loss: 1.9905 ||timer: 0.3374 sec.
Epoch: 31 || iter 2288 || Loss: 1.7919 ||timer: 0.3411 sec.
Epoch: 31 || iter 2296 || Loss: 2.5646 ||timer: 0.3343 sec.
Epoch: 31 || iter 2304 || Loss: 2.3755 ||timer: 0.3331 sec.
Epoch: 31 || iter 2312 || Loss: 1.7657 ||timer: 0.3304 sec.
Epoch: 31 || iter 2320 || Loss: 2.7140 ||tim

KeyboardInterrupt: 

In [ ]:
'''step_index = 0
for iteration in range(START_ITER, cfg['max_iter']):
    if iteration in cfg['lr_steps']:
        step_index += 1
        adjust_learning_rate(optimizer, GAMMA, step_index)
    
    # make sure data iter not out of range
    try:
        images, targets = next(batch_iterator)
        #print(targets[0][0][4].item(), label[int(targets[0][0][4].item())])
    except StopIteration:
        batch_iterator = iter(data_loader)
        images, targets = next(batch_iterator)
    if CUDA:
        images = Variable(images.cuda())
        targets = [Variable(ann.cuda(), volatile=True) for ann in targets]
    else:
        images = Variable(images)
        targets = [Variable(ann, volatile=True) for ann in targets]
    
    # Forward
    t0 = time.time()
    out = net(images)
    # backprop
    optimizer.zero_grad()
    loss_l, loss_c = criterion(out, targets)
    loss = loss_l + loss_c
    loss.backward()
    optimizer.step()
    t1 = time.time()
    loc_loss += loss_l.item()
    conf_loss += loss_c.item()
    
    if iteration % 10 == 0:
            print('timer: %.4f sec.' % (t1 - t0))
            print('iter ' + repr(PRETRAINED_ITER + iteration) + ' || Loss: %.4f ||' % (loss.item()), end='')
    
    if iteration != 0 and iteration % SAVE_MODEL_ITER == 0:
            print('Saving state, iter:', iteration)
            torch.save(ssd_net.state_dict(), SAVE_FOLDER + DATASET_NAME + "_" +
                       repr(PRETRAINED_ITER + iteration) + '.pth')
# Save final model
torch.save(ssd_net.state_dict(),
            SAVE_FOLDER + DATASET_NAME + '.pth')'''

In [ ]:
torch.cuda.empty_cache()